In [1]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 24.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 19.9 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
results_dataset = "/home/aditya/.cache/kagglehub/datasets/martj42/international-football-results-from-1872-to-2017/versions/93"
ranking_dataset = "/home/aditya/.cache/kagglehub/datasets/cashncarry/fifaworldranking/versions/15"

import pandas as pd

former_names = pd.read_csv(f"{results_dataset}/former_names.csv")
goalscorers = pd.read_csv(f"{results_dataset}/goalscorers.csv")
results = pd.read_csv(f"{results_dataset}/results.csv")
shootouts = pd.read_csv(f"{results_dataset}/shootouts.csv")

former_names.head()
# goalscorers.head()
# results.head()
# shootouts.head()

,current,former,start_date,end_date
0,Benin,Dahomey,1959-11-08,1975-11-30
1,Burkina Faso,Upper Volta,1960-04-14,1984-08-04
2,Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
3,Czechoslovakia,Bohemia,1903-04-05,1919-01-01
4,Czechoslovakia,Bohemia and Moravia,1939-01-01,1945-05-01


In [6]:
def match_result(row):
    if row["home_score"] > row["away_score"]:
        return "home_win"
    elif row["home_score"] < row["away_score"]:
        return "away_win"
    else:
        return "draw"

results["match_result"] = results.apply(match_result, axis=1)

results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,match_result
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,draw
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,home_win
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,home_win
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,draw
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,home_win


In [10]:
ranking = pd.read_csv(f"{ranking_dataset}/fifa_ranking-2024-06-20.csv")
ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,140.0,Brunei Darussalam,BRU,2.0,0.0,140,AFC,1992-12-31
1,33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31
2,32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31
3,31.0,Greece,GRE,38.0,0.0,31,UEFA,1992-12-31
4,30.0,Algeria,ALG,39.0,0.0,30,CAF,1992-12-31


In [13]:
ranking["rank_date"] = pd.to_datetime(ranking["rank_date"])
results["date"] = pd.to_datetime(results["date"])

home_rankings = ranking.rename(columns={
    "country_full": "home_team",
    "rank": "home_rank",
    "rank_date": "date"
})

away_rankings = ranking.rename(columns={
    "country_full": "away_team",
    "rank": "away_rank",
    "rank_date": "date"
})

In [14]:
results = results.sort_values("date")
home_rankings = home_rankings.sort_values("date")
away_rankings = away_rankings.sort_values("date")

results = pd.merge_asof(results, home_rankings[["date", "home_team", "home_rank"]],
                        on="date", by="home_team", direction="backward")

results = pd.merge_asof(results, away_rankings[["date", "away_team", "away_rank"]],
                        on="date", by="away_team", direction="backward")

In [15]:
results = results[results["date"] >= pd.to_datetime("1992-12-31")].copy()

In [16]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,match_result,home_rank,away_rank
18723,1993-01-01,Ghana,Mali,1,1,Friendly,Libreville,Gabon,True,draw,39.0,69.0
18724,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Libreville,Gabon,False,draw,55.0,97.0
18725,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait City,Kuwait,False,home_win,71.0,161.0
18726,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Libreville,Gabon,True,home_win,97.0,69.0
18727,1993-01-03,Gabon,Ghana,2,3,Friendly,Libreville,Gabon,False,away_win,55.0,39.0


In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Drop rows with missing rank values
results = results.dropna(subset=["home_rank", "away_rank"])

# Define features and target
features = ["home_rank", "away_rank"]  # Add other features like tournament if needed
X = results[features]
y = results["match_result"]

# One-hot encode the target and features
y = y.map({"home_win": 0, "away_win": 1, "draw": 2})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
model = xgb.XGBClassifier(use_label_encoder=False)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=["home_win", "away_win", "draw"]))

/home/aditya/.pyenv/versions/3.12.10/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:15:18] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

    home_win       0.61      0.83      0.70      2293
    away_win       0.50      0.51      0.50      1298
        draw       0.26      0.07      0.10      1101

    accuracy                           0.56      4692
   macro avg       0.46      0.47      0.44      4692
weighted avg       0.50      0.56      0.51      4692

